In [58]:
!apt -qq -y install fonts-nanum

The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 122531 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20180306-3_all.deb ...
Unpacking fonts-nanum (20180306-3) ...
Setting up fonts-nanum (20180306-3) ...
Processing triggers for fontconfig (2.13.1-2ubuntu3) ...


In [1]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fontpath = '/user/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath,size=9)
plt.rc('font', family = 'NamumBarunGothic')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import pandas as pd
naver_news = pd.read_csv("/content/BigKinds.csv", encoding='utf-8')


In [36]:
# EDA
naver_news.isnull().sum()

Local      0
Content    0
dtype: int64

In [39]:
naver_news.columns

Index(['Local', 'Content'], dtype='object')

In [31]:
naver_news = naver_news.dropna()

In [32]:
import pandas as pd
import glob, os
import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 700)

In [33]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from konlpy.tag import Okt

#불용어
stopwords = set(pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.flatten())
okt = Okt()
del_words=['중구','울산 중구','울산','김해시','김포','부천시','남구','관악구','울산 남구','서울','성동구','여주시','분당','부산진구','울산','서구','광주','광주 서구','인천','인천 서구','광산구','광주 광산구','사업','지원','개최']

def text_cleaning_11_1(text):
    # hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    # result = hangul.sub('', text)
    # 형태소 추출
    global okt
    nouns = okt.nouns(text)
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    nouns = [x for x in nouns if x not in del_words]
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    return nouns


tfidf_vectorizer = TfidfVectorizer(
    tokenizer = lambda x: text_cleaning_11_1(x),
    ngram_range=(1, 3),
    max_features=12000,
    min_df=0.03, # df 하위 5퍼센트 미만은 사용 하지 않음
    max_df=0.95,  # df 상위 95퍼센트 이상은 사용 하지 않음
    stop_words=list(stopwords)
)

# bow_vect = tfidf_vectorizer.fit_transform(naver_news['Content'].tolist()) # dt

# feature_vector = tfidf_vectorizer.fit_transform([corpus])
feature_vector = tfidf_vectorizer.fit_transform(naver_news['Content'])

word_list = tfidf_vectorizer.get_feature_names_out()
# count_list = bow_vect.toarray().sum(axis=0)

In [77]:
word_list

array(['가가', '가가 사례', '가가 호호', ..., '힙합', '힙합 스웨', '힙합 에어로빅'],
      dtype=object)

In [ ]:
# 명사, 부사, 조사 
corpus = ''.join(naver_news['Content'].tolist())

from konlpy.tag import Okt
okt = Okt()
cor_l = {term[0] for term in okt.pos(corpus, stem=True) if term[1] in ["Noun", "Verb", "Adjective"]}

import matplotlib.pyplot as plt
string_len = [len(x) for x in cor_l]
plt.hist(string_len, alpha=0.5, color='orange')

# Tfidf 기반 군집 수행

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_vectorizer = TfidfVectorizer(
#     tokenizer=my_tokenizer, # 개발자가 직접 만든 토크나이저를 전달. 토크나이징 함수는 반드시 단어가 들어있는 배열로 리턴
#     ngram_range=(1, 2),
#     min_df=0.05, # df 하위 5퍼센트 미만은 사용 하지 않음
#     max_df=0.95,  # df 상위 95퍼센트 이상은 사용 하지 않음
#     stop_words='english'
# )

# feature_vector = tfidf_vectorizer.fit_transform(doc_df['content'])

In [95]:
from sklearn.cluster import KMeans

# 3개 군집화 수행
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=42)
km_cluster.fit(feature_vector)

KMeans(max_iter=10000, n_clusters=3, random_state=42)

In [96]:
# cluster 레이블, center 좌표
cluster_labels = km_cluster.labels_
cluster_labels

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [97]:
cluster_centroids = km_cluster.cluster_centers_
cluster_centroids.shape

(3, 4)

In [98]:
# 군집 소속 정보를 dataframe에 넣어 놓기
naver_news['cluster_label'] = cluster_labels
naver_news.sort_values(by='cluster_label')

,Local,Content,cluster_label
16925,김해시,김해시 코로나가 백신 접종 적극 독려,0
60751,인천 서구,인천 서구 코로나선제적 방역으로 확대한다,0
13394,김포,속보 당국 김포일가족 코로나 재확진 바이러스 재활성 확률 높아,0
60746,인천 서구,인천 서구 검단탑병원 코로나로부터 외래입원 모두 안전한 국민안심병원 선정,0
13391,김포,코로나 재확진 김포일가족 재감염보다 재활성화 가능성,0
...,...,...,...
57158,울산 중구,울산 중구 지역사회보장협의체 위원 위촉식,2
7850,광주서구,광주 서구 지역 사회서비스투자사업 이용자 모집,2
57150,울산 중구,울산 중구 학성동 지역사회보장협독 거어르신에 건강 선식 전달,2
7892,광주서구,광주 서구 지역 주도형 청년 일자리 사업 시동,2


# 군집 별 핵심 단어 추출하기
 - centeroid에서 가장 큰 값이 가장 중심이 되는 단어

In [99]:
cluster_centroids = km_cluster.cluster_centers_
cluster_centroids

array([[1.64144600e-02, 1.98336368e-02, 2.57413662e-02, 9.74503680e-01],
       [3.94781203e-02, 3.99220097e-02, 1.32757606e-04, 4.48531795e-05],
       [4.14906094e-02, 3.33404537e-02, 9.68960808e-01, 1.65541730e-04]])

In [100]:
# cluster_model : 군집 모델
# cluster_data : 데이터 프레임
# feature_names : 단어 목록
# cluster_num : 클러스터 번호
# top_n_features : 대표 단어 몇 개를 볼 것인지 결정
def get_cluster_details(cluster_model, cluster_data, feature_names, cluster_num, top_n_features=10):

  # top_n Feature, 파일 이름, feature tfidf값
  cluster_details = {}

  # 군집 센터 좌표(centroid) 얻어내기
  cluster_centroid = cluster_model.cluster_centers_

  # cenroid 좌표의 제일 큰 값이 그 군집의 중심 단어가 되기 때문에, 정렬해서 인덱스 얻어내기(argsort)
  center_features = cluster_centroid.argsort()[:, ::-1] # 행은 내비두고 열만 내림차순으로 정렬
  
  # 각 군집 별 순환(iteration) 하면서 핵심 단어, feature의 값(실수), 각 군집에 속해있는 파일명 얻어내기
  for c_num in range(cluster_num):
    # 개별 군집 별 데이터를 담아낼 준비
    cluster_details[c_num] = {}
    cluster_details[c_num]['cluster'] = c_num # 클러스터 번호 담기

    # 단어 목록 담아주기 (top n)
    top_n_feature_indexes = center_features[c_num, :top_n_features]
    top_features = [ feature_names[idx] for idx in top_n_feature_indexes ]
    cluster_details[c_num]['top_features'] = top_features

    # 단어 중심위치(centroid) 값을 담아주기 (top n)
    top_n_feature_values = cluster_centroid[c_num, top_n_feature_indexes].tolist()
    cluster_details[c_num]['top_features_value'] = top_n_feature_values

    # # 군집에 포함된 파일명 담아주기
    # local_names = cluster_data[cluster_data['cluster_label'] == c_num]['Local']
    # local_names = local_names.tolist() # 리스트로 만들어 주기

    # cluster_details[c_num]['filenames'] = local_names
  
  return cluster_details

In [101]:
cluster_detail_info = get_cluster_details(
    cluster_model=km_cluster,
    cluster_data = naver_news,
    feature_names=tfidf_vectorizer.get_feature_names_out(),
    cluster_num=3,
    top_n_features=10
)

pd.DataFrame(cluster_detail_info)

,0,1,2
cluster,0,1,2
top_features,"[코로나, 지역, 센터, 부산]","[센터, 부산, 지역, 코로나]","[지역, 부산, 센터, 코로나]"
top_features_value,"[0.9745036796477199, 0.025741366216494324, 0.019833636819181693, 0.016414459964670027]","[0.03992200973221368, 0.03947812033382335, 0.00013275760626530736, 4.48531794734932e-05]","[0.9689608075477866, 0.041490609404477075, 0.0333404537199306, 0.00016554172999909918]"


In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

doc_list = ['사과 사과 바나나 오렌지 ' ,
            '사과 사과 바나나 오렌지 멜론',
            '이게 제가 할 일이에요? 사과']

tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = tfidf_vect_simple.fit_transform(doc_list)
print(feature_vect_simple.shape)

(3, 7)


In [17]:
feature_vect_simple.toarray()

array([[0.        , 0.47606294, 0.73941068, 0.47606294, 0.        ,
        0.        , 0.        ],
       [0.53058735, 0.40352536, 0.62674687, 0.40352536, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.32274454, 0.        , 0.54645401,
        0.54645401, 0.54645401]])

In [48]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(feature_vector[1] , feature_vector)
print(similarity_simple_pair)

[[0.0174363  1.         0.03133079 ... 0.         0.         0.        ]]


예시. 2번 클러스터와 비슷한 문서 찾기. 얼마나 비슷한지도 확인

In [19]:
cluster_2_indexes = naver_news[naver_news['cluster_label']==2].index
cluster_2_indexes

Int64Index([   3,    6,    7,   11,   13,   14,   17,   19,   20,   22,
            ...
            2303, 2308, 2314, 2315, 2316, 2318, 2319, 2322, 2324, 2325],
           dtype='int64', length=1443)

# 문서 유사도

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

cluster_2_indexes = naver_news[naver_news['cluster_label']==2].index

# 호텔로 클러스터링된 데이터 중 첫번째 문서를 추출하여 파일명 표시.  
comparison_docname = naver_news.iloc[cluster_2_indexes[0]]['Local']
print('##### 비교 기준 문서명 ',comparison_docname,' 와 타 문서 유사도######')

# 2번 클러스터 내의 첫 번째 문서와 다른 모든 문서들의 유사도 구하기
similarity_pair = cosine_similarity(feature_vector[cluster_2_indexes[2]] , feature_vector[cluster_2_indexes])
print(similarity_pair)

##### 비교 기준 문서명  경기 수원시 장안구  와 타 문서 유사도######


IndexError: ignored

In [28]:
feature_vector[cluster_2_indexes[2325]]

IndexError: ignored

In [27]:
feature_vector[cluster_2_indexes]

IndexError: ignored

In [21]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rc('font', family='AppleGothic')
plt.rc('font', family='Malgun Gothic') 
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

# argsort()를 이용하여 앞예제의 첫번째 문서와 타 문서간 유사도가 큰 순으로 정렬한 인덱스 추출하되 자기 자신은 제외. 
sorted_index = similarity_pair.argsort()[:,::-1]
sorted_index = sorted_index[:, 1:]

# 유사도가 큰 순으로 cluster_2_indexes 추출하여 재 정렬
cluster2_sorted_indexes = cluster_2_indexes[sorted_index.reshape(-1)]

# 유사도가 큰 순으로 유사도 값을 재정렬하되 자기 자신은 제외
cluster2_sim_value = np.sort(similarity_pair.reshape(-1))[::-1]
cluster2_sim_value = cluster2_sim_value[1:]

# 유사도가 큰 순으로 정렬된 인덱스와 유사도 값을 이용해 파일명과 유사도값을 막대 그래프로 시각화
cluster2_sim_df = pd.DataFrame()
cluster2_sim_df['Local'] = naver_news.iloc[cluster2_sorted_indexes]['Local']
cluster2_sim_df['similarity'] = cluster2_sim_value
print('가장 유사도가 큰 파일명 및 유사도:\n', cluster2_sim_df.iloc[0, :])

sns.barplot(x='similarity', y='Local',data=cluster2_sim_df)
plt.title(comparison_docname)


NameError: ignored